In [2]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Input
from tensorflow.keras.optimizers import RMSprop

# Download and load Shakespeare text
filepath = tf.keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)

# Read and preprocess the text
with open(filepath, 'rb') as file:
    text = file.read().decode(encoding='utf-8').lower()

# Use a subset of the text for training
text = text[300000:800000]

# Create mappings from characters to indices and vice versa
characters = sorted(set(text))
char_to_index = {c: i for i, c in enumerate(characters)}
index_to_char = {i: c for i, c in enumerate(characters)}

# Define sequence parameters
SEQ_LENGTH = 40
STEP_SIZE = 3

# Prepare input-output sequences
sentences = []
next_characters = []

for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_characters.append(text[i + SEQ_LENGTH])

# Initialize input (x) and output (y) arrays
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool)
y = np.zeros((len(sentences), len(characters)), dtype=bool)

# One-hot encode the input and output
for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[i, t, char_to_index[character]] = 1
    y[i, char_to_index[next_characters[i]]] = 1

# Build the model
model = Sequential()
model.add(Input(shape=(SEQ_LENGTH, len(characters))))  # Explicit input shape
model.add(LSTM(128))  # LSTM layer for sequence processing
model.add(Dense(len(characters)))  # Dense output layer
model.add(Activation('softmax'))  # Softmax for categorical output

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01)
)

# Train the model
model.fit(x, y, batch_size=256, epochs=4)

# Save the model in the recommended Keras format
model.save('textgenerator.keras')

print("Model trained and saved successfully.")

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 2.4796
Epoch 2/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 1.7651
Epoch 3/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 1.6073
Epoch 4/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 1.5188
Model trained and saved successfully.


In [4]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)  # Fixed typo
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, character in enumerate(sentence):
            x[0, t, char_to_index[character]] = 1

        predictions = model.predict(x, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

# Generate text with different temperature values
print('________0.2_________')
print(generate_text(300, 0.2))
print('________0.4_________')
print(generate_text(300, 0.4))
print('________0.6_________')
print(generate_text(300, 0.6))
print('________0.8_________')
print(generate_text(300, 0.8))
print('________1.0_________')
print(generate_text(300, 1.0))

________0.2_________
lord warwick, but my son,
whom i unnatured the soul and make me the soul,
and the soul and the soner's days the sears,
and the hands the soul soul some the seat,
and so the soul and the soul and to the signing the brother,
the soul some seen and the hands the first,
and that the heart the love the soul soul, and the hands the soul soul,
a
________0.4_________
elay this marriage for a month, a week;
i do the soul and that speak the servented
whose the wass in the brides of the servant.

king richard ii:
thou wears the soul happy set the forther warwick,
how now that here the bray the seat the broof and the traitors,
and no have the king richard and him that stander'd with the soul,
and for a se
________0.6_________
ashy too;
but time will not permit: all that the sinely;
how it impood hath no brother, but i have see:
now with her not, but their house,
but love the world him but was a seat youth and.

britoous:
past the sent she have baneness of hands.
thy hould to m